In [237]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import pandas as pd
import numpy as np

In [238]:
df = pd.read_csv("PCCSelected.csv")
df = pd.DataFrame(df)
X = df.drop('Outcome', axis = 1)
y = df['Outcome']

In [239]:
# Initialize XGBoost classifier
xgb_classifier = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    n_estimators=100,
    random_state=42
)

In [240]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SequentialFeatureSelector
sfs = SFS(xgb_classifier,
          k_features = 3,
          forward=True,
          scoring='accuracy',
          cv=5,
          n_jobs=-1)

In [241]:
# Fit SequentialFeatureSelector
sfs.fit(X_train, y_train)

# Get the selected feature indices as a list
selected_feature_indices = list(sfs.k_feature_idx_)
selected_feature_names = X.columns[selected_feature_indices]

In [242]:
# Use NumPy array indexing to select features from the training and testing sets
X_train_selected = X_train.iloc[:, selected_feature_indices]
X_test_selected = X_test.iloc[:, selected_feature_indices]

# Fit the XGBoost classifier with selected features
xgb_classifier.fit(X_train_selected, y_train)

# Predict on the test set
y_pred = xgb_classifier.predict(X_test_selected)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.

In [243]:

# Print the results"
print(f"Selected Feature Indices: {selected_feature_indices}")
print(f"Accuracy with {len(selected_feature_indices)} features: {accuracy*100}")
print(f"Selected Feature Names: {selected_feature_names.tolist()}")

Selected Feature Indices: [0, 2, 4]
Accuracy with 3 features: 75.32467532467533
Selected Feature Names: ['Glucose', 'BMI', 'Pregnancies']


In [245]:
final = df[['Glucose', 'BMI', 'Pregnancies']]
final['Outcome'] = df['Outcome']
final.to_csv('PCC_Boosted.csv', index = False)

C:\Users\adity\AppData\Local\Temp\ipykernel_16576\4137127890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Outcome'] = df['Outcome']
